In [9]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from google.colab import drive
from tensorflow.keras.applications import MobileNet

In [10]:
import re
import subprocess
from pathlib import Path
from typing import List, Optional

import openvino as ov
import tensorflow as tf
import tensorflow_datasets as tfds
from tqdm import tqdm

import nncf

import os
import random
import cv2
import torch
import numpy as np
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader
from tensorflow.keras.utils import to_categorical

In [13]:
# 코랩 파일 마운트
drive.mount('/content/drive')

def loadfile(path):
    X = []
    Y = []

    for label in ('danger_key', 'danger', 'danger_st', 'danger_wi','normal' ,'warnning_both','warnning_leaf','warnning_sq') :
        print("Loading training images for the label: " + label)

        for filename in os.listdir(path + label + "/"):
            img = cv2.imread(path + label + "/" + filename)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
            img[:,:,0] = cv2.equalizeHist(img[:,:,0])
            img = cv2.cvtColor(img, cv2.COLOR_YUV2BGR)
            img = cv2.resize(img, (224, 224))
            X.append(img)
            if label == 'danger':
                Y.append(0)
            elif label == 'danger_key':
                Y.append(1)
            elif label == 'danger_st':
                Y.append(2)
            elif label == 'danger_wi':
                Y.append(3)
            elif label == 'normal':
                Y.append(4)
            elif label == 'warnning_sq':
                Y.append(5)
            elif label == 'warnning_leaf':
                Y.append(6)
            elif label == 'warnning_both':
                Y.append(7)


    X = np.array(X)
    Y = np.array(Y)

    return X, Y


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# 데이터 불러오기
X, y = loadfile('/content/drive/MyDrive/data/')

# Train :test : val = 6:2:2 / set 분리
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Normalize data
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
X_val = X_val.astype('float32') / 255

# Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# Compute quantities required for featurewise normalization
datagen.fit(X_train)

Loading training images for the label: danger_key
Loading training images for the label: danger
Loading training images for the label: danger_st
Loading training images for the label: danger_wi
Loading training images for the label: normal
Loading training images for the label: warnning_both
Loading training images for the label: warnning_leaf
Loading training images for the label: warnning_sq


In [16]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.regularizers import l1_l2

# 기존 Mobilenet V2 모델 불러오기 (include_top=False는 완전 연결 계층을 포함할지 여부를 결정합니다)

# 새로운 모델 정의하기
# Build model
model = Sequential()
model.add(MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet'))  # Use MobileNet
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))  # Increase dropout
model.add(Dense(512, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))  # Add additional Dense layer, Add L1 and L2 regularization
model.add(BatchNormalization())
model.add(Dense(256, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)))   # Add additional Dense layer, Add L1 and L2 regularization
model.add(BatchNormalization())
model.add(Dense(8, activation='softmax')) # Class 조정 3 -> 8

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [17]:
# Model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_3  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_1 (Dropout)         (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 512)               655872    
                                                                 
 batch_normalization (Batch  (None, 512)               2048      
 Normalization)                                                  
                                                      

In [19]:
# Fit the model on the batches generated by datagen.flow().
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    steps_per_epoch=len(X_train) / 32,
                    epochs=100,
                    validation_data=(X_val, y_val),
                    #callbacks=[early_stopping]
                    )

model.save('/content/drive/MyDrive/231205_1430.h5')
tf.saved_model.save(model, '231205_1430')

Epoch 1/100
16/16 [==============================] - 142s 9s/step - loss: 219.5138 - accuracy: 0.2296 - val_loss: 185.0252 - val_accuracy: 0.0819
Epoch 2/100
16/16 [==============================] - 103s 6s/step - loss: 157.0576 - accuracy: 0.4650 - val_loss: 130.5753 - val_accuracy: 0.2515
Epoch 3/100
16/16 [==============================] - 105s 7s/step - loss: 109.0652 - accuracy: 0.4747 - val_loss: 90.7246 - val_accuracy: 0.2690
Epoch 4/100
16/16 [==============================] - 110s 7s/step - loss: 76.1650 - accuracy: 0.5720 - val_loss: 63.9370 - val_accuracy: 0.2982
Epoch 5/100
16/16 [==============================] - 107s 7s/step - loss: 54.4574 - accuracy: 0.5992 - val_loss: 47.2644 - val_accuracy: 0.3450
Epoch 6/100
16/16 [==============================] - 103s 6s/step - loss: 40.8396 - accuracy: 0.6498 - val_loss: 36.4382 - val_accuracy: 0.3509
Epoch 7/100
16/16 [==============================] - 100s 6s/step - loss: 32.7302 - accuracy: 0.6498 - val_loss: 30.7107 - val_accu

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
tf.saved_model.save(model,'231205_1430')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-23-5ba1e85affc7>", line 1, in <cell line: 1>
    tf.saved_model.save(model,'231205_1430')
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/saved_model/save.py", line 1331, in save
    save_and_return_nodes(obj, export_dir, signatures, options)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/saved_model/save.py", line 1373, in save_and_return_nodes
    path_helpers.get_or_create_variables_dir(export_dir)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/saved_model/path_helpers.py", line 26, in get_or_create_variables_dir
    file_io.recursive_create_dir(variables_dir)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/lib/io/file_io.py", line 498, in recursive_create_dir
    recursive_create_dir_v2(dirname)

In [ ]:
# Test Acc, Test Loss 확인

from sklearn.metrics import confusion_matrix, classification_report

# Evaluate model
score = model.evaluate(X_test, y_test)
print("Test Loss:", score[0])

print("Test Accuracy:", score[1])



In [ ]:
import time

# Measure inference time for a single sample
start_time = time.time()
sample_prediction = model.predict(np.expand_dims(X_val[0], axis=0))  # Replace X_val[0] with your sample
end_time = time.time()

inference_time = end_time - start_time
print(f"Inference time for a single sample: {inference_time} seconds")

In [ ]:
# Predict classes
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Find unique labels in test data
unique_labels = np.unique(y_test)

# Create target names from unique labels
target_names = ['class_' + str(int(i)) for i in unique_labels]

# Confusion Matrix 결과값
cm = confusion_matrix(y_test, y_pred_classes)
print("Confusion Matrix:")
print(cm)



In [ ]:
# Classification Report 분류확인 보고서
cr = classification_report(y_test, y_pred_classes, target_names=target_names)
print("Classification Report:")
print(cr)

In [8]:
import os
import re
import subprocess
from pathlib import Path
from typing import List, Optional
import openvino as ov
import tensorflow as tf
import tensorflow_datasets as tfds
from tqdm import tqdm
import nncf
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive')
#print('current:', os.getcwd())

ROOT = Path('/content/drive/MyDrive/231205_1430_model').parent.resolve()
WEIGHTS_URL = "/content/drive/MyDrive/231205_1430.h5"
DATASET_CLASSES = 8


def validate(model: ov.Model, val_loader: tf.data.Dataset) -> tf.Tensor:
    compiled_model = ov.compile_model(model)
    output = compiled_model.outputs[0]

    metric = tf.keras.metrics.CategoricalAccuracy(name="acc@1")
    for images, labels in tqdm(val_loader):
        pred = compiled_model(images.numpy())[output]
        metric.update_state(labels, pred)

    return metric.result()


def run_benchmark(model_path: str, shape: Optional[List[int]] = None, verbose: bool = True) -> float:
    command = f"benchmark_app -m {model_path} -d CPU -api async -t 15"
    if shape is not None:
        command += f' -shape [{",".join(str(x) for x in shape)}]'
    cmd_output = subprocess.check_output(command, shell=True)  # nosec
    if verbose:
        print(*str(cmd_output).split("\\n")[-9:-1], sep="\n")
    match = re.search(r"Throughput\: (.+?) FPS", str(cmd_output))
    return float(match.group(1))


def get_model_size(ir_path: str, m_type: str = "Mb", verbose: bool = True) -> float:
    xml_size = os.path.getsize(ir_path)
    bin_size = os.path.getsize(os.path.splitext(ir_path)[0] + ".bin")
    for t in ["bytes", "Kb", "Mb"]:
        if m_type == t:
            break
        xml_size /= 1024
        bin_size /= 1024
    model_size = xml_size + bin_size
    if verbose:
        print(f"Model graph (xml):   {xml_size:.3f} Mb")
        print(f"Model weights (bin): {bin_size:.3f} Mb")
        print(f"Model size:          {model_size:.3f} Mb")
    return model_size


###############################################################################
# Create a Tensorflow model and dataset


def center_crop(image: tf.Tensor, image_size: int, crop_padding: int) -> tf.Tensor:
    shape = tf.shape(image)
    image_height = shape[0]
    image_width = shape[1]

    padded_center_crop_size = tf.cast(
        ((image_size / (image_size + crop_padding)) * tf.cast(tf.minimum(image_height, image_width), tf.float32)),
        tf.int32,
    )

    offset_height = ((image_height - padded_center_crop_size) + 1) // 2
    offset_width = ((image_width - padded_center_crop_size) + 1) // 2

    image = tf.image.crop_to_bounding_box(
        image,
        offset_height=offset_height,
        offset_width=offset_width,
        target_height=padded_center_crop_size,
        target_width=padded_center_crop_size,
    )

    image = tf.compat.v1.image.resize(
        image, [image_size, image_size], method=tf.image.ResizeMethod.BILINEAR, align_corners=False
    )

    return image


def preprocess_for_eval(image, label):
    image = center_crop(image, 224, 32)
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    image = tf.image.convert_image_dtype(image, tf.float32)

    label = tf.one_hot(label, DATASET_CLASSES)

    return image, label


val_dataset = tfds.load("imagenette/320px-v2", split="validation", shuffle_files=False, as_supervised=True)
val_dataset = val_dataset.map(preprocess_for_eval).batch(128)

weights_path = tf.keras.utils.get_file("/content/drive/MyDrive/231205_1430", WEIGHTS_URL, cache_subdir="models")
tf_model = tf.keras.applications.MobileNetV2(weights=weights_path, classes=DATASET_CLASSES)

###############################################################################
# Quantize a Tensorflow model
#
# The transformation function transforms a data item into model input data.
#
# To validate the transform function use the following code:
# >> for data_item in val_loader:
# >>    model(transform_fn(data_item))


# def transform_fn(data_item):
#     images, _ = data_item
#     return images


# # The calibration dataset is a small, no label, representative dataset
# # (~100-500 samples) that is used to estimate the range, i.e. (min, max) of all
# # floating point activation tensors in the model, to initialize the quantization
# # parameters.
# #
# # The easiest way to define a calibration dataset is to use a training or
# # validation dataset and a transformation function to remove labels from the data
# # item and prepare model input data. The quantize method uses a small subset
# # (default: 300 samples) of the calibration dataset.

# calibration_dataset = nncf.Dataset(val_dataset, transform_fn)
# tf_quantized_model = nncf.quantize(tf_model, calibration_dataset)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Quantize a Tensorflow model
#
# The transformation function transforms a data item into model input data.
#
# To validate the transform function use the following code:
# >> for data_item in val_loader:
# >>    model(transform_fn(data_item))


def transform_fn(data_item):
    images, _ = data_item
    return images


# The calibration dataset is a small, no label, representative dataset
# (~100-500 samples) that is used to estimate the range, i.e. (min, max) of all
# floating point activation tensors in the model, to initialize the quantization
# parameters.
#
# The easiest way to define a calibration dataset is to use a training or
# validation dataset and a transformation function to remove labels from the data
# item and prepare model input data. The quantize method uses a small subset
# (default: 300 samples) of the calibration dataset.

calibration_dataset = nncf.Dataset(val_dataset, transform_fn)
tf_quantized_model = nncf.quantize(tf_model, calibration_dataset)

###############################################################################
# Benchmark performance, calculate compression rate and validate accuracy

ov_model = ov.convert_model(tf_model, share_weights=False)
#경량화 모델 : ov_quntized_model
ov_quantized_model = ov.convert_model(tf_quantized_model, share_weights=False)

fp32_ir_path = f"{ROOT}/mobilenet_v2_fp32.xml"
ov.save_model(ov_model, fp32_ir_path, compress_to_fp16=False)
print(f"[1/7] Save FP32 model: {fp32_ir_path}")
fp32_model_size = get_model_size(fp32_ir_path, verbose=True)

int8_ir_path = f"{ROOT}/mobilenet_v2_int8.xml"
ov.save_model(ov_quantized_model, int8_ir_path, compress_to_fp16=False)
print(f"[2/7] Save INT8 model: {int8_ir_path}")
int8_model_size = get_model_size(int8_ir_path, verbose=True)

print("[3/7] Benchmark FP32 model:")
fp32_fps = run_benchmark(fp32_ir_path, shape=[1, 224, 224, 3], verbose=True)
print("[4/7] Benchmark INT8 model:")
int8_fps = run_benchmark(int8_ir_path, shape=[1, 224, 224, 3], verbose=True)

print("[5/7] Validate OpenVINO FP32 model:")
fp32_top1 = validate(ov_model, val_dataset)
print(f"Accuracy @ top1: {fp32_top1:.3f}")

print("[6/7] Validate OpenVINO INT8 model:")
int8_top1 = validate(ov_quantized_model, val_dataset)
print(f"Accuracy @ top1: {int8_top1:.3f}")

print("[7/7] Report:")
print(f"Accuracy drop: {fp32_top1 - int8_top1:.3f}")
print(f"Model compression rate: {fp32_model_size / int8_model_size:.3f}")
# https://docs.openvino.ai/latest/openvino_docs_optimization_guide_dldt_optimization_guide.html
print(f"Performance speed up (throughput mode): {int8_fps / fp32_fps:.3f}")

INFO:nncf:Creating compression algorithm: quantization
INFO:nncf:Overflow issue fix was applied to first convolution weight quantizers.
INFO:nncf:Collecting tensor statistics/data |█████           | 1 / 3
INFO:nncf:Collecting tensor statistics/data |██████████      | 2 / 3
INFO:nncf:Collecting tensor statistics/data |████████████████| 3 / 3
INFO:nncf:BatchNorm statistics adaptation |█████           | 1 / 3
INFO:nncf:BatchNorm statistics adaptation |██████████      | 2 / 3
INFO:nncf:BatchNorm statistics adaptation |████████████████| 3 / 3
[1/7] Save FP32 model: /content/drive/MyDrive/mobilenet_v2_fp32.xml
Model graph (xml):   0.079 Mb
Model weights (bin): 8.397 Mb
Model size:          8.476 Mb
[2/7] Save INT8 model: /content/drive/MyDrive/mobilenet_v2_int8.xml
Model graph (xml):   0.397 Mb
Model weights (bin): 2.242 Mb
Model size:          2.639 Mb
[3/7] Benchmark FP32 model:
[ INFO ] Count:            1452 iterations
[ INFO ] Duration:         15035.20 ms
[ INFO ] Latency:
[ INFO ]    

100%|██████████| 31/31 [01:21<00:00,  2.65s/it]


Accuracy @ top1: 0.198
[6/7] Validate OpenVINO INT8 model:


100%|██████████| 31/31 [01:21<00:00,  2.64s/it]

Accuracy @ top1: 0.198
[7/7] Report:
Accuracy drop: 0.000
Model compression rate: 3.211
Performance speed up (throughput mode): 1.143


INFO:nncf:Creating compression algorithm: quantization
INFO:nncf:Overflow issue fix was applied to first convolution weight quantizers.
INFO:nncf:Collecting tensor statistics/data |█████           | 1 / 3
INFO:nncf:Collecting tensor statistics/data |██████████      | 2 / 3
INFO:nncf:Collecting tensor statistics/data |████████████████| 3 / 3
INFO:nncf:BatchNorm statistics adaptation |█████           | 1 / 3
INFO:nncf:BatchNorm statistics adaptation |██████████      | 2 / 3
INFO:nncf:BatchNorm statistics adaptation |████████████████| 3 / 3
[1/7] Save FP32 model: /content/drive/MyDrive/mobilenet_v2_fp32.xml
Model graph (xml):   0.079 Mb
Model weights (bin): 8.397 Mb
Model size:          8.476 Mb
[2/7] Save INT8 model: /content/drive/MyDrive/mobilenet_v2_int8.xml
Model graph (xml):   0.397 Mb
Model weights (bin): 2.242 Mb
Model size:          2.639 Mb
[3/7] Benchmark FP32 model:
[ INFO ] Count:            1302 iterations
[ INFO ] Duration:         15035.50 ms
[ INFO ] Latency:
[ INFO ]    

100%|██████████| 31/31 [01:22<00:00,  2.65s/it]


Accuracy @ top1: 0.198
[6/7] Validate OpenVINO INT8 model:


100%|██████████| 31/31 [00:53<00:00,  1.72s/it]

Accuracy @ top1: 0.198
[7/7] Report:
Accuracy drop: 0.000
Model compression rate: 3.211
Performance speed up (throughput mode): 1.253


In [ ]:
|